# Livrable 1 : Classification Binaire
|Erwan Martin|Thibaut Liger-Hellard|Arnaud Maturel|Guillaume Le Cocguen|Victorien Goudeau|
|------------|---------------------|--------------|--------------------|-----------------|

**Contexte**:
L'objectif de ce livrable est classifier les photos des "non-photos" parmis les images du dataset.
La classification sera donc binaire.


# import des librairies

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os

import tensorflow as tf
import tensorboard


In [2]:
#verify GPU available (not mendatory)
print("GPUs Available: ", tf.config.list_physical_devices('GPU'))

GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Préparation des données
Afin de faire la classification binaire des images il va falloir préparer nos données en étiquettant nos images et en les mettant a la même échelle (scaling).

In [74]:
#data path definition
f = open('../data/path.txt', "r")
DATAPATH = f.read()
print(DATAPATH)

D:\CESI\A5\datascience\Projet\data


In [91]:
# dataset creation
labels = []
imgs = []

#creating labels table -----------------------------
for i in os.listdir(path=DATAPATH):
    if os.path.isdir(DATAPATH+"/"+i):
        labels.append(i)
#end creating labels table -----------------------------

#removing livrable2  from dataset------------------------
labels.remove("Dataset_L2")
labels.remove("Photo_2")
#end removing livrable2 from dataset------------------------


print(f"LABELS : {labels}")

#create BIG csv with image path + label
csv = open("../data/dataset_L1.csv", "w")
csv.write("pathname;label\n")
for label in labels:
    if label == "Photo":
        l = 1
    else:
        l=0
    for img in os.listdir(path=DATAPATH+"/"+label):
        csv.write(DATAPATH+"/"+label+"/"+img+";"+str(l)+"\n")
csv.close()

LABELS : ['Painting', 'Photo', 'Schematics', 'Sketch', 'Text']
